#### Profiling algorithm

In [7]:
import cProfile
import pstats
from pstats import SortKey

from utils import get_comparison_table
from vns import basic_VNS

In [8]:
SOLUTIONS_FILE = '/home/ubuntu/VNS_for_UMApHMP/data/AP/Solutions-UMApHMP.txt'
DATASET = 'AP'
INPUT_DIRECTORY = "./data/AP/generated/"

Let's profile `get_comparison_table` since it calls vns multiple times, for NUMBER_OF_PROBLEMS problems, so we have average statistics for different problems out of the box.

In [9]:
NUMBER_OF_PROBLEMS = 10
cProfile.run('comparison_table = get_comparison_table([basic_VNS], NUMBER_OF_PROBLEMS, DATASET, INPUT_DIRECTORY, SOLUTIONS_FILE)', 'output/profile_stats')
p = pstats.Stats('output/profile_stats')
p.strip_dirs().sort_stats('cumtime').print_stats(30)

100%|██████████| 10/10 [00:45<00:00,  4.59s/it]

Sun Aug 13 11:11:23 2023    output/profile_stats

         7019867 function calls (6985675 primitive calls) in 46.037 seconds

   Ordered by: cumulative time
   List reduced from 576 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      8/1    0.000    0.000   46.037   46.037 {built-in method builtins.exec}
        1    0.000    0.000   46.037   46.037 <string>:1(<module>)
        1    0.001    0.001   46.037   46.037 utils.py:228(get_comparison_table)
       10    0.007    0.001   45.892    4.589 vns.py:117(basic_VNS)
    17050    0.056    0.000   45.745    0.003 utils.py:40(__init__)
    17050    0.050    0.000   45.689    0.003 utils.py:53(_get_cost)
    17050    0.152    0.000   45.639    0.003 utils.py:170(get_solution_cost_fw)
      260    0.007    0.000   45.198    0.174 vns.py:73(local_search_best_improvement)
      533    0.044    0.000   45.111    0.085 vns.py:77(<listcomp>)
    17050   23.390    0.001   23.390    0.001 u

Let's take a look at the interesting ones:

```
ncalls  tottime  percall  cumtime  percall   filename:lineno(function)
1         0.001    0.001   48.364   48.364   utils.py:212(get_comparison_table)
10        0.006    0.001   48.235    4.823   vns.py:117(basic_VNS)
17124     26.206   0.002   47.984    0.003   utils.py:137(get_solution_cost_fw)
260       0.007    0.000   47.512    0.183   vns.py:73(local_search_best_improvement)
17124     0.079    0.000   18.209    0.001   graph_utils.py:41(floyd_warshall)
17124     0.076    0.000    6.110    0.000   _base.py:348(asformat)
17124     0.439    0.000    6.004    0.000   _coo.py:372(tocsr)
17124     0.847    0.000    3.097    0.000   {scipy.sparse.csgraph._shortest_path.floyd_warshall}
17124     1.178    0.000    1.491    0.000   utils.py:70(get_discount_matrix)
17124     0.083    0.000    1.473    0.000   _validation.py:9(validate_graph)
17124     1.065    0.000    1.084    0.000   utils.py:99(_get_valid_cost_graph)
260       0.001    0.000    0.687    0.003   vns.py:106(shake)
260       0.002    0.000    0.686    0.003   vns.py:102(_shake_simple)
794       0.003    0.000    0.074    0.000   utils.py:56(get_neighbourhood)
544       0.038    0.000    0.071    0.000   utils.py:167(get_swap_neighbourhood)
40        0.000    0.000    0.016    0.000   utils.py:67(get_distance_matrix)
10        0.000    0.000    0.001    0.000   vns.py:14(get_initial_solution_robust)

```

How do we explain this line?
```
534    0.042    0.000   47.426    0.089 vns.py:77(<listcomp>)
```

It's this line:

> neighbourhood = [Solution(neighbour, solution.problem) for neighbour in solution.get_neighbourhood(neighbourhood_type)]

Costs are calculated here implicitly, when creating Solution - it's intentionally like that so the cost is always calulated only once per solution.

Check this:

```
17124    0.079    0.000   18.209    0.001 graph_utils.py:41(floyd_warshall)
17124    0.439    0.000    6.004    0.000 _coo.py:372(tocsr)
17124    0.847    0.000    3.097    0.000 {scipy.sparse.csgraph._shortest_path.floyd_warshall}
```

```
def floyd_warshall(graph):
    sparse_graph = csr_matrix(graph)
    _, predecessors = csgraph.floyd_warshall(csgraph=sparse_graph, directed=True, return_predecessors=True)
    return predecessors
```

`scipy.sparse.csgraph._shortest_path.floyd_warshall` itself takes significatnly less time compared to preparing the right data format for the input.

We can definitelly say that it's only worth improving `get_solution_cost_fw` at this point, since it's taking 99.5 % of VNS time.

### Let's than profile `get_solution_cost_fw` in more details (we extracted parts of it in separate functions for this purpose)

In [ ]:
NUMBER_OF_PROBLEMS = 30
cProfile.run('comparison_table = get_comparison_table([basic_VNS], NUMBER_OF_PROBLEMS, DATASET, INPUT_DIRECTORY, SOLUTIONS_FILE)', 'output/profile_stats')

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [40:13<00:00, 80.46s/it] 


In [ ]:
p = pstats.Stats('output/profile_stats')
p.strip_dirs().sort_stats('cumtime').print_stats(30)

Sun Aug 13 10:52:32 2023    output/profile_stats

         74127320 function calls (73816246 primitive calls) in 2413.813 seconds

   Ordered by: cumulative time
   List reduced from 344 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000 2413.813 2413.813 {built-in method builtins.exec}
        1    0.000    0.000 2413.813 2413.813 <string>:1(<module>)
        1    0.002    0.002 2413.813 2413.813 utils.py:228(get_comparison_table)
       30    0.038    0.001 2413.652   80.455 vns.py:117(basic_VNS)
   155534    0.699    0.000 2412.266    0.016 utils.py:40(__init__)
   155534    0.722    0.000 2411.567    0.016 utils.py:53(_get_cost)
   155534    1.807    0.000 2410.845    0.016 utils.py:170(get_solution_cost_fw)
      782    0.037    0.000 2405.663    3.076 vns.py:73(local_search_best_improvement)
     1626    0.611    0.000 2404.967    1.479 vns.py:77(<listcomp>)
   155534 2024.024    0.013 2024.024    0.0

For the first 10 problems:
```
17143    0.154    0.000   45.916  =100%   0.003   utils.py:170(get_solution_cost_fw)          
17143   23.622    0.001   23.622  =51%    0.001   utils.py:145(_normal_paths_calulation)        3*n^2
17143    0.087    0.000   18.048  =39%    0.001   graph_utils.py:41(floyd_warshall)             p*n^2  ???
17143    0.090    0.000    2.724  =0.1%   0.000   utils.py:137(_prepare)                        2*n^2
17143    0.443    0.000    1.369  =0.02%  0.000   utils.py:160(_peculiar_paths_calulations)     p*n
```

For the first 25 problems:
```
100783    1.068    0.000 1057.147  =100%   0.010   utils.py:170(get_solution_cost_fw)
100783  854.984    0.008  854.984  =80.7%  0.008   utils.py:145(_normal_paths_calulation)
100783    0.568    0.000  118.567  =11.2%  0.001   graph_utils.py:41(floyd_warshall)
100783    0.633    0.000   61.356  =5%     0.001   utils.py:137(_prepare)
100783    6.718    0.000   21.172  =1%     0.000   utils.py:160(_peculiar_paths_calulations)

```

For the first 30 problems:
```
155534    1.807    0.000 2410.845  =100%  0.016   utils.py:170(get_solution_cost_fw)
155534 2024.024    0.013 2024.024  =83%   0.013 utils.py:145(_normal_paths_calulation)
155534    0.978    0.000  194.008  =8%    0.001 graph_utils.py:41(floyd_warshall)
155534    1.028    0.000  147.759         0.001 utils.py:137(_prepare)
155534   13.680    0.000   43.249         0.000 utils.py:160(_peculiar_paths_calulations)
```

This is weird! If they are of the same complexity, how come `_normal_paths_calulation` have such increase in time with increase of n??